# PROGRES - Mini-Projet 3
# DHT Chord 

Fabien Mathieu - fabien.mathieu@normalesup.org

Sébastien Tixeuil - Sebastien.Tixeuil@lip6.fr

Le but de ce mini-projet est de construire un simulateur de DHT pour tester ses performances:
construction de la table, gestion distribuée du contenu, analyse de la répartition de charge et du trafic.

La table servira à gérer des pages Wikipedia. De manière **exceptionnelle**, le pair en charge d'une page sera également responsable de la récupération et de la mise à disposition du contenu de la page. On rappelle que ce n'est pas le comportement typique d'une DHT, qui est seulement supposée pointer vers les pairs possédant le contenu. Cette surcharge est introduite ici afin de réduire la complexité d'implantation du mini-projet.

# Exercice 1. Création d'une classe Peer

Réaliser en Python une classe `Peer` permettant de simuler une DHT Chord.

La communication sera simulée au travers d'un dictionnaire `dht`: `hash` -> `Peer` : connaître le hash d'un pair permet de le contacter (dans la vraie vie, en plus du hash c'est l'adresse IP qui est connue), et les appels réseaux au sein de la DHT seront représentés par des appels à la clé de la destination dans le dictionnaire.

La fonction de hash à utiliser est SHA1, disponible sur la librairy `hashlib` https://docs.python.org/3/library/hashlib.html

SHA1 fournit des hash de 160 bits.

Les instances de la classe `Peer` devront implanter (au moins) les attributs / méthodes suivantes:
- `id` : identifiant du pair (entier)
- `hash` : le hash de l'identifiant du pair
- `predecessor` : le hash du prédécesseur du pair
- `successor` : le hash du successeur du pair
- `finger`: la finger table du pair (cf cours)
- `update_finger(hash_list)`: une fonction qui utilise une liste de hashs de pairs pour mettre à jour la finger table, i.e. ajouter des sauts manquants ou remplacer des sauts existants par de meilleurs sauts.
- `give_neighbors()`: une fonction qui retourne la liste des hash connus (son propre hash, celui du prédecesseur, du successeur, ceux de la `finger`)
- `refresh_finger()`: Une fonction qui rafraichit la finger table en demandant à chaque voisin la liste de ses voisins
- `isincharge(key)`: une fonction qui indique si le pair est en charge de `key`
- `lookup(key, dht)`: une fonction qui renvoie le hash du pair en charge de `key` ainsi que la distance, en nombre de sauts, qui sépare le pair qui effectue le *lookup* du pair responsable
- `table`: voir plus bas 
- `publish_page(page_title, dht)`: voir plus bas
- `retrieve_page(page_title)`: voir plus bas
- `search(search_string, dht)`: voir plus bas

L'attribut `table` contient le dictionnaire des clés gérées par le pair ; la table pourra contenir deux types de clés :
  - des clés de mot : à un mot donné on associe le hash du mot préfixé par `word:` (par exemple `computer` est représenté par le hash SHA1 de `word:computer`). La valeur associée dans la table est un ensemble (`set` Python), qui va typiquement contenir les titres de pages wikipedia qui contiennent le mot.
  - des clés de page : à une page wikipedia donnée on associe le hash du titre de la page préfixé par `page:` (par exemple `Circuit (computer science)` est représenté par le hash SHA1 de `page:Circuit (computer science)`). La valeur associée dans la table est le contenu de la page.

La méthode `publish_page` devra réaliser les actions suivantes :
- Séparer le titre en mots écrits en minuscules. Par exemple, `Circuit (computer science)` devient `["circuit", "computer", "science"]`, `Object-oriented programming` devient `["object", "oriented", "programming"]`.
- Pour chaque mot, trouver le pair responsable de la clé de mot. Si cette clé existe, rajouter le titre de la page à l'ensemble associé, sinon initialiser la clé (avec comme valeur un ensemble à un élément, le titre de la page). N'oubliez pas que la mise à jour se fait chez le pair responsable, pas chez le pair qui appelle la méthode.

La méthode `publish_page` n'est pas en charge de récupérer le contenu de la page proprement dite.

La méthode `retrieve_page` est responsable de fournir le contenu. Elle devra réaliser les actions suivantes :
- Si le pair est responsable de la page, renvoyer le contenu:
  - Si la clé est présente dans la table, renvoyer la valeur associée
  - Si la clé n'est pas présente, utiliser le module wikipedia pour récupérer la page à partir du titre (méthode `wikipedia.page`), stocker le contenu (attribut `content` du résultat) dans la table et le renvoyer
  - En cas d'erreur du module wikipedia (cela peut se produire, mais assez rarement), ne rien stocker et renvoyer `"Page not found"`
- Si le pair n'est pas responsable de la page, récupérer le contenu auprès du responsable de la page.

La méthode `search` devra : 
- Séparer la recherche en mots écrits en minuscules.
- Récupérer pour chaque mot l'ensemble des pages Wikipedia contenant le mot (si le mot n'est pas indexé, c'est l'ensemble vide)
- Renvoyer la liste des titres contenant tous les mots de la recherche.
- Si la liste est vide mais qu'au moins un mot est indexé, émettre un warning (cf https://docs.python.org/3/library/logging.html) et renvoyer l'union (titres contenant au moins un des mots de la recherche).
- Si aucun mot n'est indexé, émettre un warning signalant qu'aucun des mots n'a été trouvé et renvoyer une liste vide.

L'initialisation d'un pair prendra trois arguments :
- L'indentifiant (entier)
- La DHT (le dictionnaire `dht`)
- L'identifiant du point d'entrée

Si la DHT est vide, le pair se rajoute simplement dedans (c'est son propre prédécesseur et successeur, et sa finger est vide). Sinon, il s'insère via le point d'entrée, comme vu en cours et en TP.

**Remarque** : `table`, `publish_page`, `retrieve_page` et `search` ne seront utilisés qu'à partir de l'exercice 3. Vous pouvez ércire une première version qui ne contient pas ces attributs / méthodes, et mettre à jour au moment de l'exercice 3.

# Exercice 2. Création de la DHT

À partir d'une DHT vide, insérer des 10000 pairs d'identifiant 0 à 9999 dans cet ordre, en utilisant 0 comme point d'insertion.
- Quelle est la taille moyenne des fingers? La taille de la plus petite finger ? Celle de la plus grande ?
- À partir du pair d'identifiant 42, identifier grâce à la méthode `lookup` la distance (en sauts) qui le sépare des responsables de chacun des 10000 pairs. Quelle est la distance moyenne ? La plus grande distance ?
- Faire exécuter à chaque pair la méthode `refresh_finger` et refaire les deux questions précédentes. Que constatez-vous ?
- Répéter la question précédente. Que constatez-vous ?

# Exercice 3. Insertion d'un corpus Wikipedia

En utilisant le module wikipedia et la méthode `links` des objets `WikipediaPage`, récupérer l'ensemble des pages situées à distance au plus 2 de la page `Computer science`, c'est-à-dire :
- `Computer science`
- Les titres des pages référencés par la page `Computer science`
- Les titres des pages référencés par les pages référencés par la page `Computer science`
- Après avoir éliminé les doublons, sauver la liste dans un fichier (pour pouvoir la ré-utiliser plus tard) et publier tous les titres dans la DHT. Vous pourrez utiliser un pair arbitraire pour appeler la méthode.

- Combien de mots sont indexés au total ?
- Quelle est la taille moyenne des tables des pairs ? La taille de la plus grande table ?

# Exercice 4. Utilisation de la DHT

- Commencer par tester manuellement le fonctionnement de la DHT : à partir d'un pair quelconque (par exemple d'identifiant 42), effectuer quelques recherches (`search`) et récupérer quelques contenus (`retrieve_page`). Ne pas hésiter à tester des recherches partielles (intersection vide) et des recherches sans succès.
- De manière programmatique, récupérer grâce à la dht à partir d'un pair quelconque le contenu de 2000 pages parmi celles publiées (vous pourrez utiliser la liste des titres que vous avez sauvegardée). Quel est le plus grand nombre d'articles indexé par un pair?